# Job Queue runbook

In [ ]:
%%bash
docker run --name queue -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 postgres:16.2
sleep 4

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE IF NOT EXISTS job_queue (
    id BIGINT PRIMARY KEY,
    job_data TEXT NOT NULL
);

"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

INSERT INTO job_queue (id, job_data)
    VALUES  (1, 'Task 1'),
            (2, 'Task 2'),
            (3, 'Task 3'),
            (4, 'Task 4'),
            (5, 'Task 5'),
            (6, 'Task 6'),
            (7, 'Task 7'),
            (8, 'Task 8'),
            (9, 'Task 9'),
            (10,'Task 10');

"

In [ ]:
import psycopg
from psycopg.abc import Query

def query(sql: Query) -> list[tuple[str, str]] | None:
    with psycopg.connect("host=localhost dbname=postgres user=postgres") as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            return cur.fetchall()

In [ ]:
# Get one job from the queue
query("""
    SELECT id, job_data
    FROM job_queue
    LIMIT 1
""")


In [ ]:
# Get one job from the queue, from the start
query("""
    SELECT id, job_data
    FROM job_queue
    ORDER BY id
    LIMIT 1
""")


In [ ]:
# Get one job from the queue, from the start, and lock it
query("""
    SELECT id, job_data
    FROM job_queue
    ORDER BY id
    LIMIT 1
    FOR UPDATE
""")


In [ ]:
# Get one job from the queue, from the start, and lock it and don't deadlock
query("""
    SELECT id, job_data
    FROM job_queue
    ORDER BY id
    LIMIT 1
    FOR UPDATE
    SKIP LOCKED
""")


In [ ]:
%%bash
docker stop queue
docker rm queue